In [6]:
# %pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 5.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
from pyspark import SparkConf, pandas as ps
from pyspark.sql import SparkSession, functions as sf
from pyspark.sql.functions import col, collect_list
from pyspark.sql.types import *
import psycopg2
import time

In [45]:
conf = SparkConf().set("spark.driver.memory", "8g")

spark_session = SparkSession\
    .builder\
    .master("local")\
    .config(conf=conf)\
    .appName("OLTP_population") \
    .getOrCreate()

In [85]:
# schema = StructType([
#     StructField("Transaction_ID", IntegerType(), True),
#     StructField("Date", DateType(), True),
#     StructField("Customer_Name", StringType(), True),
#     StructField("Product", ArrayType(StringType()), True),
#     StructField("Total_Items", IntegerType(), True),
#     StructField("Total_Cost", DecimalType(), True),
#     StructField("Payment_Method", StringType(), True),
#     StructField("City", StringType(), True),
#     StructField("Store_Type", StringType(), True),
#     StructField("Discount_Applied", BooleanType(), True),
#     StructField("Customer_Category", StringType(), True),
#     StructField("Season", StringType(), True),
#     StructField("Promotion", StringType(), True)
# ])


df = spark_session.read.csv('Retail_Transactions_Dataset.csv', header=True, inferSchema=True, quote='"')
df.head()


Row(Transaction_ID=1000000000, Date=datetime.datetime(2022, 1, 21, 6, 27, 29), Customer_Name='Stacey Price', Product="['Ketchup', 'Shaving Cream', 'Light Bulbs']", Total_Items=3, Total_Cost=71.65, Payment_Method='Mobile Payment', City='Los Angeles', Store_Type='Warehouse Club', Discount_Applied=True, Customer_Category='Homemaker', Season='Winter', Promotion='None')

In [84]:
# df.Product = df.Product.cast(ArrayType(StringType()))

In [52]:
customers = df[['Customer_Name', 'Customer_Category']]

In [53]:
customers.head(10)

[Row(Customer_Name='Stacey Price', Customer_Category='Homemaker'),
 Row(Customer_Name='Michelle Carlson', Customer_Category='Professional'),
 Row(Customer_Name='Lisa Graves', Customer_Category='Professional'),
 Row(Customer_Name='Mrs. Patricia May', Customer_Category='Homemaker'),
 Row(Customer_Name='Susan Mitchell', Customer_Category='Young Adult'),
 Row(Customer_Name='Joshua Frazier', Customer_Category='Retiree'),
 Row(Customer_Name='Victoria Garrett', Customer_Category='Student'),
 Row(Customer_Name='Sydney Waller', Customer_Category='Young Adult'),
 Row(Customer_Name='Kimberly Morgan', Customer_Category='Homemaker'),
 Row(Customer_Name='Lori Conway', Customer_Category='Young Adult')]

In [115]:
 # = sf.split(df.Product,',')
# print(products)
# prods = [sf.split(p, ',').alias("Product") for p in products]
# print(prods)
tran_prod = df.select("Transaction_ID", sf.explode(sf.split(sf.regexp_replace(df.Product, "^\\[|'+|\\]$",''),',')).alias("Product"))
tran_prod.head(10)


[Row(Transaction_ID=1000000000, Product='Ketchup'),
 Row(Transaction_ID=1000000000, Product=' Shaving Cream'),
 Row(Transaction_ID=1000000000, Product=' Light Bulbs'),
 Row(Transaction_ID=1000000001, Product='Ice Cream'),
 Row(Transaction_ID=1000000001, Product=' Milk'),
 Row(Transaction_ID=1000000001, Product=' Olive Oil'),
 Row(Transaction_ID=1000000001, Product=' Bread'),
 Row(Transaction_ID=1000000001, Product=' Potatoes'),
 Row(Transaction_ID=1000000002, Product='Spinach'),
 Row(Transaction_ID=1000000003, Product='Tissues')]

In [117]:
tran = df.select("Transaction_ID", "Date", "Customer_Name", "Total_Cost", "Payment_Method", "City", "Store_Type", "Discount_Applied", "Promotion")
tran.head(10)

[Row(Transaction_ID=1000000000, Date=datetime.datetime(2022, 1, 21, 6, 27, 29), Customer_Name='Stacey Price', Total_Cost=71.65, Payment_Method='Mobile Payment', City='Los Angeles', Store_Type='Warehouse Club', Discount_Applied=True, Promotion='None'),
 Row(Transaction_ID=1000000001, Date=datetime.datetime(2023, 3, 1, 13, 1, 21), Customer_Name='Michelle Carlson', Total_Cost=25.93, Payment_Method='Cash', City='San Francisco', Store_Type='Specialty Store', Discount_Applied=True, Promotion='BOGO (Buy One Get One)'),
 Row(Transaction_ID=1000000002, Date=datetime.datetime(2024, 3, 21, 15, 37, 4), Customer_Name='Lisa Graves', Total_Cost=41.49, Payment_Method='Credit Card', City='Houston', Store_Type='Department Store', Discount_Applied=True, Promotion='None'),
 Row(Transaction_ID=1000000003, Date=datetime.datetime(2020, 10, 31, 9, 59, 47), Customer_Name='Mrs. Patricia May', Total_Cost=39.34, Payment_Method='Mobile Payment', City='Chicago', Store_Type='Pharmacy', Discount_Applied=True, Promoti

AttributeError: 'DataFrame' object has no attribute 'select'

In [12]:
db_params = {
    'host': 'rfm-segmentation-oltp-db-1',
    'database': 'trans_oltp',
    'user': 'admin',
    'password': 'password',
    'port': '5432'
}


In [20]:
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()
cursor.execute(';')
output = cursor.fetchone()
print(output)
conn.close()

UndefinedFunction: function tables() does not exist
LINE 1: SELECT tables();
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.
